In [35]:
import pandas as pd
import os
import re
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats
from em_utils import *

In [36]:
# set thresholds
max_iter = 400
n_est = 100
n_js = 1
OOB = True
mrf = MERF(fixed_effects_model =
           RandomForestRegressor(n_estimators = n_est, 
                                n_jobs = n_js, 
                                oob_score= True),
            gll_early_stop_threshold=None,
            max_iterations=max_iter)

cv = KFold(n_splits=5, shuffle=True)
mse_rf = []
mse_rfplus = []
mse_mrf = []
mse_mrf_id = []
mse_mrf_id_both = []

Useful Functions 

In [ ]:
# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_plots"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/"
os.makedirs(output_dir, exist_ok=True)

print("---------- Read metadata ----------")
m1_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/clinical/transformed/aim2"
test = read_data(m1_dir, "a2_test_samples_standard_clinical.csv")
train = read_data(m1_dir, "a2_train_samples_standard_clinical.csv")
full = read_data(m1_dir, "a2_meta_Transformed_standard_clinical.csv")
full_raw = read_data(m1_dir, "a2_meta_not_Transformed_standard_clinical.csv")

print(full_raw.columns.to_list() == train.columns.to_list())
print(train.columns.to_list() == test.columns.to_list())

Make long format 

In [ ]:
# Apply the function to each meta dataset
print("---------- Convert metadata to long format ----------")
full_long = make_long(full_raw)
full_long['x_t'] = full_long['subject_id'].astype(str) + '.' + full_long['time'].astype(str)

train_long = make_long(train)
train_long['x_t'] = train_long['subject_id'].astype(str) + '.' + train_long['time'].astype(str)

test_long = make_long(test)
test_long['x_t'] = test_long['subject_id'].astype(str) + '.' + test_long['time'].astype(str)

In [ ]:
print("train data outcome_BMI_fnl values:", train_long['outcome_BMI_fnl'])
print("Full columns after transformation:", full_long.columns.to_list())
print("Train columns after transformation:", train_long.columns.to_list())
print("Test columns after transformation:", test_long.columns.to_list())

Drop columns only if they exist in the DataFrame (since some may not be present after merge)

In [40]:
columns_to_drop = ['Unnamed: 0', 'cohort_number', 'record_id', 'x_t']
full_long = full_long.drop([col for col in columns_to_drop if col in full.columns], axis=1)
train_long = train_long.drop([col for col in columns_to_drop if col in train_long.columns], axis=1)
test_long = test_long.drop([col for col in columns_to_drop if col in test_long.columns], axis=1)

In [ ]:
# Check the final columns
print("Final columns after drop:", full_long.columns.to_list())
print("Final test columns after drop:", test_long.columns.to_list())

In [42]:
# Drop NA 
test_long = test_long.dropna()
train_long = train_long.dropna()
full_long = full_long.dropna()
raw_train = full_long[full_long['subject_id'].isin(train_long['subject_id'])]
raw_test = full_long[full_long['subject_id'].isin(test_long['subject_id'])]

In [ ]:
print("---------- raw train and tax ----------")
print("raw_train shape = ", raw_train.shape)
print("raw_test shape = ", raw_test.shape)
print("---------- preprocessed train and tax ----------")
print("test_long shape = ", test_long.shape)
print("train_long shape = ", train_long.shape)

In [ ]:
print("---------- Select predictors for Basic Raw training set ----------")
train_set = raw_train
X = train_set.drop(['outcome_BMI_fnl', 'subject_id'], axis=1)
#X = X.drop(columns=['Unnamed: 0_tax', 'x_t'], errors='ignore')
Y = train_set[['outcome_BMI_fnl']]
Y = Y['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array
clusters_train = train_set['subject_id'].to_numpy() # Get ID variables
Z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones
time = train_set['time'].astype(float).to_numpy() # Get time values as numeric array 

# Check the final columns
print("Final columns after drop:", X.columns.to_list())
print("X train values:", train_set['outcome_BMI_fnl'])

print("---------- Select predictors for Basic Raw test set ----------")
test_set = raw_test
X_new = test_set.drop(['outcome_BMI_fnl', 'subject_id'], axis=1)
X_new = X_new[X.columns]  # Reorder and select columns to match training set
X_new = X_new.astype(X.dtypes)  # Ensure data types match

Y_new = test_set['outcome_BMI_fnl'].to_numpy()  # Convert Y to numeric array
clusters_new = pd.Series(test_set['subject_id'])  # Convert to pandas Series
# Create random effects matrix with ones
Z_new = np.ones((len(X_new), 1))
time_new = test_set['time'].astype(float).to_numpy()  # Convert time values to numeric array

In [45]:
# print("---------- RUN MERF RAW BASIC🌱 ----------")
# mrf_basic = mrf.fit(X.select_dtypes(include=[np.number]), 
#         Z, 
#         pd.Series(clusters_train), 
#         Y)

In [46]:
# plot_merf_training_stats(mrf_basic)
# plt.savefig(os.path.join(output_dir, '1.clinical/merf_raw_metrics.png'), dpi=300, bbox_inches='tight')


In [47]:
# Predict using the fitted model
# X_new = X_new.drop(columns=['x_t'], errors='ignore')
# y_hat_new = mrf_basic.predict(X_new, Z_new, clusters_new)

In [48]:
# Calculate and print RMSE and R-squared
# print("---------- Metrics for MERF BASIC🌱 ----------")
# calculate_metrics(Y_new, y_hat_new)

In [49]:
# forest = mrf_basic.trained_fe_model
# oob = str(round(forest.oob_score_*100, 1))  # percent variation
# print(f" % Variation in training set: {oob}")
# print(percent_var_statement(step="full_forest", oob=oob))

In [50]:
# # Save all the components of forest to a csv file
# df = pd.DataFrame.from_dict(forest.__dict__, orient='index')
# df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs"
# df.to_csv(os.path.join(df_dir,'merf_basic_raw_forest_components.csv'))

R - squared version for mixed model, PREV maybe 

In [51]:
# Usage of the featur importance function
# print("---------- Feature importances for MERF BASIC🌱 ----------")
# plot_feature_importances(forest.feature_names_in_, 
#                          forest.feature_importances_, 
#                          output_dir, 'merf_raw_feature_importances.png')


In [52]:
# plot_predicted_vs_actual(Y_new, y_hat_new, output_dir, 'clinical_predicted_vs_actual.png')

In [ ]:
print("---------- RUN MERF preprocessed with participant RE 🌱🌸 ----------")
 # Mixed Effects Random Forest Training with participant RE and time cluster 
train_set = train_long 
X_train = train_set.drop(['outcome_BMI_fnl', 'subject_id', 'time'], axis=1).to_numpy()
Z_train = np.array((np.ones(len(train_set)), train_set['subject_id'].apply(lambda s: int(s[-3:])))).T
clusters_train = pd.Series(train_set['subject_id'].apply(lambda s: int(s[-3:]))).astype(float)  # Convert to float if necessary
y_train = train_set[['outcome_BMI_fnl']]
y_train = y_train['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array

In [ ]:
print(f"Dimensions of X_train: {X_train.shape}")
print(f"Dimensions of Z_train: {Z_train.shape}")
print(f"Number of unique inputs for clusters_train: {clusters_train.nunique()}")
print(f"Inputs to clusters_train: {clusters_train}")
print(f"Dimensions of y_train: {y_train.shape}")

In [55]:
# mrf_id_fe = mrf.fit(X_train, Z_train, clusters_train, y_train)

In [56]:
# forest = mrf_id_fe.trained_fe_model
# oob = str(round(forest.oob_score_*100, 1)) 
# print(f" % Variation in training set: {oob}")
# # Save all the components of forest to a csv file
# # Convert forest.__dict__ to a DataFrame with a single row
# df = pd.DataFrame.from_dict(forest.__dict__, orient='index')
# df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs"
# df.to_csv(os.path.join(df_dir,'merf_id_forest_components.csv'))

In Mixed Effects Random Forests (MERF), the Generalized Log-Likelihood (GLL) is used to evaluate the quality of the model at each iteration

In [57]:
# plot_merf_training_stats(mrf_id_fe)

Test MERF if RE on test data

In [58]:
# Test data (repeat similar steps)
test_set = test_long
X_test = test_set.drop(['outcome_BMI_fnl', 'subject_id', 'time'], axis=1).to_numpy()
Z_test = np.array((np.ones(len(test_set)), test_set['subject_id'].apply(lambda s: int(s[-3:])))).T
clusters_test = pd.Series(test_set['subject_id'].apply(lambda s: int(s[-3:]))).astype(float)  # Convert to float if necessary
# Make predictions
# yhat_merf = mrf_id_fe.predict(X_test, Z_test, clusters_test)

In [ ]:
# Calculate and print RMSE and R-squared
print("---------- Metrics for MERF preprocessed with participant RE 🌱🌸 ----------")
#calculate_metrics(test_set['outcome_BMI_fnl'], yhat_merf)

Fine Tuning basic. 

In [60]:

# print("---------- MERF with finetuning RE 🌱 ----------")
# import itertools 
# print("---------- Select predictors for Basic Raw training set ----------")
# train_set = raw_train
# X = train_set.drop(['outcome_BMI_fnl', 'subject_id', 'x_t'], axis=1)
# print(f"X dimensions: {X.shape}")
# y = train_set[['outcome_BMI_fnl']]
# y = y['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array
# print(f"y dimensions: {y.shape}")
# clusters = train_set['subject_id'].to_numpy() # Get ID variables
# print(f"clusters dimensions: {clusters.shape}")
# z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones
# print(f"z dimensions: {z.shape}")

# # Hyperparameters to tune
# param_grid = {
#     'n_estimators': [1, 100, 300],
#     'max_depth': [None, 1, 10],
#     'min_samples_split': [2, 5, 10],
#     'max_iter': [2, 30, 10]
# }

# best_score = float('inf')
# best_params = {}

# # K-fold cross-validation
# kf = KFold(n_splits=10)
# total_combinations = len(list(itertools.product(*param_grid.values())))
# current_combination = 0
# for params in itertools.product(*param_grid.values()):
#     current_combination += 1
#     print(f"Processing combination {current_combination}/{total_combinations}: {params}")
#     n_estimators, max_depth, min_samples_split, max_iter = params
#     scores = []
#     for train_index, test_index in kf.split(X):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
#         y_train, y_test = y[train_index], y[test_index]
#         clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
#         z_train, z_test = z[train_index], z[test_index]
#         #z_train = z[train_index]  # Slice z to match the size of X_train
#         # Initialize MERF with current parameters from the grid
#         # Initialize MERF with current parameters from the grid
#         model = MERF(fixed_effects_model =
#                      RandomForestRegressor(n_estimators=n_estimators, 
#                                           max_depth=max_depth, 
#                                           min_samples_split=min_samples_split,
#                                           n_jobs=n_js, 
#                                           oob_score=True),
#                      gll_early_stop_threshold=None,
#                      max_iterations=max_iter)

#         #model.fit(X_train, y_train, clusters_train)
#         model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
#         y_pred = model.predict(X_test, z_test, clusters_test)
#         scores.append(np.mean((y_pred - y_test) ** 2)) # MSE

#     # Calculate the mean of the scores for the current combination of parameters
#     mean_score = np.mean(scores)
#     # Check if the mean score is better than the current best score
#     if mean_score < best_score:
#         # Update the best score and the parameters that achieved it
#         best_score = mean_score
#         best_params = params

# print("Best parameters:", best_params)
# print("Best score:", best_score)

Best parameters: (5, 10, 5, 10)
Best score: 15.420998172500717

In [61]:
# tuned_model = MERF(fixed_effects_model =
#                 RandomForestRegressor(n_estimators=5, 
#                                         max_depth=10, 
#                                         min_samples_split=5,
#                                         n_jobs=n_js, 
#                                         oob_score=True),
#                      gll_early_stop_threshold=None,
#                      max_iterations=10)

#         #model.fit(X_train, y_train, clusters_train)
# tuned_model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
# y_pred = tuned_model.predict(X_test, z_test, clusters_test)
# mse = ((y_pred - y_test) ** 2) # MSE

In [62]:
# calculate_metrics(y_test, y_pred)

With 100 iteractions: 
Correlation between actual and predicted values: 0.3893
Root Mean Squared Error: 3.0546
R-squared Score: 0.1046

With 10 iterations:
Correlation between actual and predicted values: 0.4391
Root Mean Squared Error: 2.9560
R-squared Score: 0.1614

In [63]:
# plot_predicted_vs_actual(y_test, y_pred, output_dir, 'fine_tuned_actual_vs_predicted_10.png')

In [64]:
# forest = tuned_model.trained_fe_model
# feature_names = forest.feature_names_in_
# feature_importances = forest.feature_importances_
# plot_feature_importances(feature_names, feature_importances, output_dir, 'tuned_feature_importances_clinical')

Fine Tuning wth PREV and PTEV.
ptev is calculated as the ratio of fixed effect variance to total variance, while prev is calculated as the ratio of random effect variance to total variance.

A higher PTEV is often preferable because it suggests that your model explains a larger portion of the observed variability. However, excessively high PTEV could signal overfitting, where the model memorizes rather than generalizes.

The optimal PREV depends on your data and goals. If your model is designed to capture group- or subject-level variability, a higher PREV might be desirable. However, in most cases, excessively high PREV might indicate the model isn't capturing key fixed effects, potentially leading to underfitting.

In [ ]:
print("---------- MERF with finetuning RE 🌱 ----------")
import itertools 
import pandas as pd

# Define the directory where the CSV file will be saved
df_dir = 'path/to/your/directory'

print("---------- Select predictors for Basic Raw training set ----------")
train_set = raw_train
X = train_set.drop(['outcome_BMI_fnl', 'subject_id', 'x_t'], axis=1)
print(f"X dimensions: {X.shape}")
y = train_set[['outcome_BMI_fnl']]
y = y['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array
print(f"y dimensions: {y.shape}")
clusters = train_set['subject_id'].to_numpy() # Get ID variables
print(f"clusters dimensions: {clusters.shape}")
z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones
print(f"z dimensions: {z.shape}")

# # Hyperparameters to tune
# param_grid = {
#     'n_estimators': [1, 100, 300],
#     'max_depth': [None, 1, 100],
#     'min_samples_split': [2, 10],
#     'max_iter': [2, 30, 100]
# }

# best_score = float('inf')
# best_params = {}

# # K-fold cross-validation
# kf = KFold(n_splits=10)
# total_combinations = len(list(itertools.product(*param_grid.values())))
# current_combination = 0

# # Initialize a list to store the results of each iteration
# results = []

# for params in itertools.product(*param_grid.values()):
#     current_combination += 1
#     print(f"Processing combination {current_combination}/{total_combinations}: {params}")
#     n_estimators, max_depth, min_samples_split, max_iter = params
#     scores = []
#     for train_index, test_index in kf.split(X):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
#         y_train, y_test = y[train_index], y[test_index]
#         clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
#         z_train, z_test = z[train_index], z[test_index]

#         model = MERF(fixed_effects_model =
#                      RandomForestRegressor(n_estimators=n_estimators, 
#                                           max_depth=max_depth, 
#                                           min_samples_split=min_samples_split,
#                                           n_jobs=n_js, 
#                                           oob_score=True),
#                      gll_early_stop_threshold=None,
#                      max_iterations=max_iter)

#         model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
#         y_pred = model.predict(X_test, z_test, clusters_test)
#         scores.append(np.mean((y_pred - y_test) ** 2)) # MSE

#         # Calculate ptev and prev
#         total_variance = np.var(y_test) #calculates the total variance of the predicted values
#         random_effect_variance = np.var(y_test - y_pred)  # Variance of residuals
#         fixed_effect_variance = total_variance - random_effect_variance

#         ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
#         prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

#         # Print ptev and prev for the current iteration
#         print(f"Combination {current_combination}, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}")

#         # Calculate the mean of the scores for the current combination of parameters
#         mean_score = np.mean(scores)
#         mean_prev = np.mean(prev)
#         mean_ptev = np.mean(ptev)
#         if mean_score < best_score:
#             best_score = mean_score
#             best_params = params

#         # Append the results of the current iteration to the results list
#         # Create a result dictionary with individual scores and mean scores
#         result_dict = {
#             'n_estimators': n_estimators,
#             'max_depth': max_depth,
#             'min_samples_split': min_samples_split,
#             'max_iter': max_iter,
#             'mse_score_1': scores[0] if len(scores) > 0 else None,
#             'mse_score_2': scores[1] if len(scores) > 1 else None,
#             'mse_score_3': scores[2] if len(scores) > 2 else None,
#             'mse_score_4': scores[3] if len(scores) > 3 else None,
#             'mse_score_5': scores[4] if len(scores) > 4 else None,
#             'mse_score_6': scores[5] if len(scores) > 5 else None,
#             'mse_score_7': scores[6] if len(scores) > 6 else None,
#             'mse_score_8': scores[7] if len(scores) > 7 else None,
#             'mse_score_9': scores[8] if len(scores) > 8 else None,
#             'mse_score_10': scores[9] if len(scores) > 9 else None,
#             'mean_mse_score': mean_score,
#             'prev_1': prev[0] if len(prev) > 0 else None,
#             'prev_2': prev[1] if len(prev) > 1 else None,
#             'prev_3': prev[2] if len(prev) > 2 else None,
#             'prev_4': prev[3] if len(prev) > 3 else None,
#             'prev_5': prev[4] if len(prev) > 4 else None,
#             'prev_6': prev[5] if len(prev) > 5 else None,
#             'prev_7': prev[6] if len(prev) > 6 else None,
#             'prev_8': prev[7] if len(prev) > 7 else None,
#             'prev_9': prev[8] if len(prev) > 8 else None,
#             'prev_10': prev[9] if len(prev) > 9 else None,
#             'mean_prev': mean_prev,
#             'ptev_1': ptev[0] if len(ptev) > 0 else None,
#             'ptev_2': ptev[1] if len(ptev) > 1 else None,
#             'ptev_3': ptev[2] if len(ptev) > 2 else None,
#             'ptev_4': ptev[3] if len(ptev) > 3 else None,
#             'ptev_5': ptev[4] if len(ptev) > 4 else None,
#             'ptev_6': ptev[5] if len(ptev) > 5 else None,
#             'ptev_7': ptev[6] if len(ptev) > 6 else None,
#             'ptev_8': ptev[7] if len(ptev) > 7 else None,
#             'ptev_9': ptev[8] if len(ptev) > 8 else None,
#             'ptev_10': ptev[9] if len(ptev) > 9 else None,
#             'mean_ptev': mean_ptev
#         }
#         # Append the result dictionary to the results list
#         results.append(result_dict)

# print("Best parameters:", best_params)
# print("Best score:", best_score)

# # Convert the results list to a DataFrame and save it to a CSV file
# df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/1.clinical"
# results_df = pd.DataFrame(results)
# results_df.to_csv(f'{df_dir}/fine_tuning_results_raw_clinical.csv', index=False)
# Hyperparameters to tune
param_grid = {
    'n_estimators': [1, 5, 100, 300],
    'max_depth': [None, 1, 50, 100],
    'min_samples_split': [2, 5],
    'max_iter': [2, 3, 10, 20]
}

best_score = float('inf')
best_params = {}

# Initialize a list to store the results of each iteration
results = []

# K-fold cross-validation
kf = KFold(n_splits=10)
total_combinations = len(list(itertools.product(*param_grid.values())))
current_combination = 0
for params in itertools.product(*param_grid.values()):
        current_combination += 1
        print(f"Combination {current_combination}/{total_combinations}: {params}\n")
        n_estimators, max_depth, min_samples_split, max_iter = params
        scores = []
        prev = []
        ptev = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
            y_train, y_test = y[train_index], y[test_index]
            clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
            z_train, z_test = z[train_index], z[test_index]
            model = MERF(
                # Specify the fixed effects model as a Random Forest Regressor
                fixed_effects_model=RandomForestRegressor(
                    n_estimators=n_estimators,  # Number of trees in the forest
                    max_depth=max_depth,  # Maximum depth of each tree
                    min_samples_split=min_samples_split,  # Minimum samples required to split an internal node
                    n_jobs=1,  # Number of jobs to run in parallel
                    oob_score=True  # Whether to use out-of-bag samples to estimate the R^2 on unseen data
                ),
                # Generalized Linear Model (GLM) early stopping threshold
                gll_early_stop_threshold=None,  # No early stopping threshold set
                # Maximum number of iterations for the MERF algorithm
                max_iterations=max_iter  # Maximum number of iterations to run the MERF algorithm
            )
            #model.fit(X_train, y_train, clusters_train)
            model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
            y_pred = model.predict(X_test, z_test, clusters_test)
            scores.append(np.mean((y_pred - y_test) ** 2)) # MSE
            
            # Calculate ptev and prev
            total_variance = np.var(y_test) #calculates the total variance of the predicted values
            random_effect_variance = np.var(y_test - y_pred)  # Variance of residuals
            fixed_effect_variance = total_variance - random_effect_variance

            ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
            prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

            # Print ptev and prev for the current iteration
            print(f"Combination {current_combination}, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}")

        # Calculate the mean of the scores for the current combination of parameters
        mean_score = np.mean(scores)
        mean_prev = np.mean(prev)
        mean_ptev = np.mean(ptev)
        if mean_score < best_score:
            best_score = mean_score
            best_params = params

        # Append the results of the current iteration to the results list
        # Create a result dictionary with individual scores and mean scores
        result_dict = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'max_iter': max_iter,
            'mse_score_1': scores[0] if len(scores) > 0 else None,
            'mse_score_2': scores[1] if len(scores) > 1 else None,
            'mse_score_3': scores[2] if len(scores) > 2 else None,
            'mse_score_4': scores[3] if len(scores) > 3 else None,
            'mse_score_5': scores[4] if len(scores) > 4 else None,
            'mse_score_6': scores[5] if len(scores) > 5 else None,
            'mse_score_7': scores[6] if len(scores) > 6 else None,
            'mse_score_8': scores[7] if len(scores) > 7 else None,
            'mse_score_9': scores[8] if len(scores) > 8 else None,
            'mse_score_10': scores[9] if len(scores) > 9 else None,
            'mean_mse_score': mean_score,
            'prev_1': prev[0] if len(prev) > 0 else None,
            'prev_2': prev[1] if len(prev) > 1 else None,
            'prev_3': prev[2] if len(prev) > 2 else None,
            'prev_4': prev[3] if len(prev) > 3 else None,
            'prev_5': prev[4] if len(prev) > 4 else None,
            'prev_6': prev[5] if len(prev) > 5 else None,
            'prev_7': prev[6] if len(prev) > 6 else None,
            'prev_8': prev[7] if len(prev) > 7 else None,
            'prev_9': prev[8] if len(prev) > 8 else None,
            'prev_10': prev[9] if len(prev) > 9 else None,
            'mean_prev': mean_prev,
            'ptev_1': ptev[0] if len(ptev) > 0 else None,
            'ptev_2': ptev[1] if len(ptev) > 1 else None,
            'ptev_3': ptev[2] if len(ptev) > 2 else None,
            'ptev_4': ptev[3] if len(ptev) > 3 else None,
            'ptev_5': ptev[4] if len(ptev) > 4 else None,
            'ptev_6': ptev[5] if len(ptev) > 5 else None,
            'ptev_7': ptev[6] if len(ptev) > 6 else None,
            'ptev_8': ptev[7] if len(ptev) > 7 else None,
            'ptev_9': ptev[8] if len(ptev) > 8 else None,
            'ptev_10': ptev[9] if len(ptev) > 9 else None,
            'mean_ptev': mean_ptev
        }
        
        # Append the result dictionary to the results list
        results.append(result_dict)

print("Best parameters:", best_params)
print("Best score:", best_score)

# Convert the results list to a DataFrame and save it to a CSV file
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/1.clinical"
results_df = pd.DataFrame(results)
results_df.to_csv(f'{df_dir}/fine_tuning_results_raw_clinical.csv', index=False)


In [67]:
# Convert the results list to a DataFrame and save it to a CSV file
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/1.clinical"
results_df = pd.DataFrame(results)
results_df.to_csv(f'{df_dir}/fine_tuning_results_raw_clinical_new.csv', index=False)

In [68]:
print("Best parameters:", best_params)
print("Best score:", best_score)

Best parameters: (100, 50, 5, 10)
Best score: 15.267048924651743
